In [343]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [344]:
job_desc_ca_all = pd.read_csv('../../../data/raw_data/jack_scrape_canada.csv')
job_desc_ca_all = job_desc_ca_all.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1)

job_desc_ca_jr = pd.read_csv('../../../data/raw_data/juniorCanadaPostings.csv')
job_desc_ca_jr = job_desc_ca_jr.drop(['Unnamed: 0'], axis=1)

job_desc_ca_sr = pd.read_csv('../../../data/raw_data/seniorCanadaPostings.csv')
job_desc_ca_sr = job_desc_ca_sr.drop(['Unnamed: 0'], axis=1)

# Merge all Canada job descriptions
job_desc_ca = job_desc_ca_all.append(job_desc_ca_jr, sort=True)
job_desc_ca = job_desc_ca.append(job_desc_ca_sr, sort=True)

# Drop nans and duplicates
job_desc_ca = job_desc_ca.dropna()
job_desc_ca = job_desc_ca.drop_duplicates(subset=['Company', 'Position'])
job_desc_ca = job_desc_ca.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [345]:
job_desc_us = pd.read_csv('../../../data/raw_data/elisa_us_cleaned.csv')
job_desc_us = job_desc_us.drop(['Unnamed: 0'], axis=1)
job_desc_us = job_desc_us.replace(to_replace='NF', value=float('nan'))
job_desc_us = job_desc_us.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [346]:
job_desc_ca = job_desc_ca.rename({'Company': 'company', 'Position': 'position', 'Type': 'company_type',
                                 'Size': 'company_size'}, axis=1)
job_desc_us = job_desc_us.rename({'Company': 'company', 'Position': 'position', 'Type': 'company_type', 
                                 'Size': 'company_size'}, axis=1)

In [347]:
company_info_ca = pd.read_csv('../../../data/raw_data/companies_ca_info.csv')
company_info_ca = company_info_ca.drop(['Unnamed: 0'], axis=1)
company_info_ca = company_info_ca.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [348]:
company_info_us = pd.read_csv('../../../data/raw_data/companies_us_info.csv')
company_info_us = company_info_us.drop(['Unnamed: 0', 'index'], axis=1)
company_info_us = company_info_us.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [349]:
def get_unique_vals(column_name):
    unique_1 = job_desc_us[column_name].unique()
    unique_1 = [x for x in unique_1 if str(x) != 'nan']
    unique_2 = job_desc_ca[column_name].unique()
    unique_2 = [x for x in unique_2 if str(x) != 'nan']
    unique_3 = company_info_us[column_name].unique()
    unique_3 = [x for x in unique_3 if str(x) != 'nan']
    unique_4 = company_info_ca[column_name].unique()
    unique_4 = [x for x in unique_4 if str(x) != 'nan']
    
    results = [unique_1, unique_2, unique_3, unique_4]
    results_union = set().union(*results)
    return (unique_1, unique_2, unique_3, unique_4, results_union)

In [350]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

In [351]:
def replace_col_with_categorical(df, col_name, cat):
    df = df.drop(col_name, axis=1)
    df[cat.columns] = cat
    return df

In [352]:
def convert_to_categorical(df, column_name, unique_col_vals, unique_vals):
    company = pd.get_dummies(df[column_name], prefix=column_name)
    dif = Diff(unique_col_vals, unique_vals)
    df = replace_col_with_categorical(df, column_name, company)
    for col in dif:
        df[col] = 0
    return df

In [353]:
def convert_all_dfs_to_categorical(column_name):
    unique_1, unique_2, unique_3, unique_4, results_union = get_unique_vals(column_name)
    job_us = convert_to_categorical(job_desc_us, column_name, unique_1, results_union)
    job_ca = convert_to_categorical(job_desc_ca, column_name, unique_2, results_union)
    company_us = convert_to_categorical(company_info_us, column_name, unique_3, results_union)
    company_ca = convert_to_categorical(company_info_ca, column_name, unique_4, results_union)
    return (job_us, job_ca, company_us, company_ca)

### Type, Size, Industry -> convert to categorical

In [354]:
job_desc_us['company_size'].unique()

array(['10000+ Employees', '501 to 1000 Employees', '1 to 50 Employees',
       '201 to 500 Employees', nan, '5001 to 10000 Employees',
       '1001 to 5000 Employees', '51 to 200 Employees'], dtype=object)

In [355]:
job_desc_ca['company_size'].unique()

array(['501 to 1000 Employees', '1001 to 5000 Employees',
       '201 to 500 Employees', nan, '51 to 200 Employees',
       '10000+ Employees', '1 to 50 Employees', '5001 to 10000 Employees'],
      dtype=object)

In [357]:
job_desc_us, job_desc_ca, company_info_us, company_info_ca = convert_all_dfs_to_categorical('company_size')

In [365]:
job_desc_ca.columns

Index(['company', 'Description', 'Glassdoor Salary Estimate', 'Industry',
       'Location', 'position', 'Rating', 'Revenue', 'Sector', 'Year Founded',
       'company_size_1 to 50 Employees', 'company_size_10000+ Employees',
       'company_size_1001 to 5000 Employees',
       'company_size_201 to 500 Employees',
       'company_size_5001 to 10000 Employees',
       'company_size_501 to 1000 Employees',
       'company_size_51 to 200 Employees'],
      dtype='object')